# Исследование надежности заемщиков.

## Обзор данных

In [64]:
import pandas as pd

In [65]:
data_frame=pd.read_csv('/datasets/data.csv')
data_frame.info()
data_frame.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


## Предобработка данных

### Заполнение пропусков

In [66]:
Null_data_columns=data_frame.isna().sum() 
columns_data=list(data_frame.columns) 
display(Null_data_columns)
Null_data_columns=list(Null_data_columns) 

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Оценка количества пропущенных значений

In [67]:
print('Доля пропусков в столбце:') 
for i in range(len(columns_data)): # цикл для расчета доли пропусков в каждом столбце
    rezul=1-(len(data_frame)-Null_data_columns[i])/len(data_frame)
    print(columns_data[i], '=', f'{rezul:.2%}' )

Доля пропусков в столбце:
children = 0.00%
days_employed = 10.10%
dob_years = 0.00%
education = 0.00%
education_id = 0.00%
family_status = 0.00%
family_status_id = 0.00%
gender = 0.00%
income_type = 0.00%
debt = 0.00%
total_income = 10.10%
purpose = 0.00%


Доля пропусков в столбце ` days_employed` и `total_income` более 10%, что достаточно много

In [68]:
mean_total_income=data_frame.groupby('income_type')['total_income'].mean()
data_frame['total_income']=data_frame['total_income'].fillna(0) 
name_income=list(mean_total_income.index.unique()) 

for i in range(len(name_income)):  
    for it in range(len(data_frame)):
        if data_frame['income_type'][it]==name_income[i] and data_frame['total_income'][it]==0: 
            data_frame['total_income'][it]=mean_total_income[i]

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


### Проверка данных на аномалии и исправления.

In [69]:
for i in columns_data:
    colum_value=data_frame[i].sort_values().unique() # Вывод всех уникальных значений  по столбцам
    print('столбец',i)
    display(colum_value)

столбец children


array([-1,  0,  1,  2,  3,  4,  5, 20])

столбец days_employed


array([-18388.94990057, -17615.56326563, -16593.47281726, ...,
       401715.81174889, 401755.40047533,             nan])

столбец dob_years


...

array([0, 1])

столбец total_income


array([  20667.26379327,   21205.28056622,   21367.64835649, ...,
       1726276.01433167, 2200852.2102589 , 2265604.02872274])

столбец purpose


array(['автомобили', 'автомобиль', 'высшее образование',
       'дополнительное образование', 'жилье',
       'заняться высшим образованием', 'заняться образованием',
       'на покупку автомобиля', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля', 'на проведение свадьбы',
       'недвижимость', 'образование', 'операции с жильем',
       'операции с коммерческой недвижимостью',
       'операции с недвижимостью', 'операции со своей недвижимостью',
       'покупка жилой недвижимости', 'покупка жилья',
       'покупка жилья для сдачи', 'покупка жилья для семьи',
       'покупка коммерческой недвижимости', 'покупка недвижимости',
       'покупка своего жилья', 'получение высшего образования',
       'получение дополнительного образования', 'получение образования',
       'приобретение автомобиля', 'профильное образование',
       'ремонт жилью', 'свадьба', 'свой автомобиль',
       'сделка с автомобилем', 'сделка с подержанным автомобилем',
       'строительство 

In [70]:
data_frame['days_employed']=abs(data_frame['days_employed'])
group_days_employed=data_frame.groupby('income_type')['days_employed'].max() 
display(group_days_employed)


income_type
безработный        395302.838654
в декрете            3296.759962
госслужащий         15193.032201
компаньон           17615.563266
пенсионер          401755.400475
предприниматель       520.848083
сотрудник           18388.949901
студент               578.751554
Name: days_employed, dtype: float64

Анамально большие значения стажа выявлены у `безработный` и `пенсионер`

In [71]:
for i in range(len(data_frame)): 
    if (data_frame['days_employed'][i]>18250):
        data_frame['days_employed'][i]='NaN'
group_days_employed=data_frame.groupby('income_type')['income_type'].count() 
display(group_days_employed) 
group_days_employed=data_frame.groupby('income_type')['days_employed'].mean() 
display(group_days_employed)


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


income_type
безработный            2
в декрете              1
госслужащий         1459
компаньон           5085
пенсионер           3856
предприниматель        2
сотрудник          11119
студент                1
Name: income_type, dtype: int64

income_type
безработный                NaN
в декрете          3296.759962
госслужащий        3399.896902
компаньон          2111.524398
пенсионер                  NaN
предприниматель     520.848083
сотрудник          2324.895056
студент             578.751554
Name: days_employed, dtype: float64

**Вывод** Из полученных данных видно, что данных в группе  `безработный `,  `в декрете ` `предприниматель `,`студент` очень мало. Пропуски в данных строках ни как не должны повлиять на ответы на вопросы. А вот `пенсионер ` имеет достаточное количество данных и все они имеют не верное значение стажа, но ошибки в значении стажа не окажут влияния на ответ на поставленные вопросы. Поэтому заменим их нулями.

In [72]:
data_frame['days_employed']=data_frame['days_employed'].fillna(0)
Null_data_columns=data_frame.isna().sum() 
display(Null_data_columns)  

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Изменение типов данных.

In [73]:
data_frame['total_income']=data_frame['total_income'].astype('int')
data_frame['children']=abs(data_frame['children'])

In [74]:
data_frame['education'] = data_frame['education'].str.lower()
data_frame.duplicated().sum()

71

In [75]:
data_frame = data_frame.drop_duplicates().reset_index(drop=True)  
data_frame.duplicated().sum()

0

### Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.

In [76]:
df_education=data_frame[['education','education_id']] #словари значений
df_education= df_education.drop_duplicates().reset_index(drop=True)
display(df_education)
df_family=data_frame[['family_status','family_status_id']]
df_family= df_family.drop_duplicates().reset_index(drop=True)
display(df_family)

,education,education_id
0,высшее,0
1,среднее,1
2,неоконченное высшее,2
3,начальное,3
4,ученая степень,4


,family_status,family_status_id
0,женат / замужем,0
1,гражданский брак,1
2,вдовец / вдова,2
3,в разводе,3
4,Не женат / не замужем,4


### Категоризация дохода.

In [77]:
def income_category (data_frame): #функция для категоризации дахода
    if data_frame<=30000:
        return 'E' 
    elif data_frame >30001 and data_frame <=50000:
        return 'D'
    elif data_frame >50001 and data_frame <=200000:
        return 'C'
    elif data_frame >200001 and data_frame <=1000000:
        return 'B'
    else:
        return 'A'

data_frame['total_income_category']=data_frame['total_income'].apply(income_category) #Примение функции
data_frame.head(10) #проверка результата 

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,B
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,C
2,0,5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,C
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,B
4,0,0.000000,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,C
5,0,926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,B
6,0,2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,B
7,0,152.779569,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,C
8,2,6929.865299,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,C
9,0,2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,C


### Категоризация целей кредита.

In [78]:

def purpose_category(data_purpose): #функция по определению категории кредита 
    if  'авто' in data_purpose:
        return 'операции с автомобилем'               
    elif  'свадьб' in data_purpose:
        return 'проведение свадьбы'
    elif 'образован' in data_purpose:
        return 'получение образования'
    else:
        return ('операции с недвижимостью')
    
data_frame['purpose_category']=data_frame['purpose'].apply(purpose_category)
display(data_frame.head(10))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,B,операции с недвижимостью
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,C,операции с автомобилем
2,0,5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,C,операции с недвижимостью
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,B,получение образования
4,0,0.000000,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,C,проведение свадьбы
5,0,926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,B,операции с недвижимостью
6,0,2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,B,операции с недвижимостью
7,0,152.779569,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,C,получение образования
8,2,6929.865299,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,C,проведение свадьбы
9,0,2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,C,операции с недвижимостью


Подсчет доли людей с задолжностями к людям без задолжности в зависимости от количества детей.

In [79]:
grouped_debt = data_frame.pivot_table(index='children', columns = 'debt', values='family_status', aggfunc='count') 
grouped_debt = grouped_debt.fillna(0)
value_children=data_frame['children'].sort_values().unique() 
print('Доля должников в зависимости от количества детей:')  
for i in value_children: 
    rezul=grouped_debt[1][i]/(grouped_debt[1][i]+grouped_debt[0][i])
    print('количество детей',i, '    доля должников=', f'{rezul:.2%}')


Доля должников в зависимости от количества детей:
количество детей 0     доля должников= 7.54%
количество детей 1     доля должников= 9.17%
количество детей 2     доля должников= 9.45%
количество детей 3     доля должников= 8.18%
количество детей 4     доля должников= 9.76%
количество детей 5     доля должников= 0.00%
количество детей 20     доля должников= 10.53%


In [80]:
grouped_debt = data_frame.pivot_table(index='family_status', columns = 'debt', values='income_type', aggfunc='count') 
value_family_status=data_frame['family_status'].sort_values().unique() 

for i in value_family_status: 
    rezul=grouped_debt[1][i]/(grouped_debt[1][i]+grouped_debt[0][i])
    print('статус',i, '    доля должников=', f'{rezul:.2%}')

статус Не женат / не замужем     доля должников= 9.75%
статус в разводе     доля должников= 7.11%
статус вдовец / вдова     доля должников= 6.57%
статус гражданский брак     доля должников= 9.35%
статус женат / замужем     доля должников= 7.55%


In [81]:
grouped_income = data_frame.pivot_table(index='total_income_category', columns = 'debt', values='income_type', aggfunc='count')
value_family_status=data_frame['total_income_category'].sort_values().unique()  
for i in value_family_status:
    rezul=grouped_income[1][i]/(grouped_income[1][i]+grouped_income[0][i])
    print('категория',i, '    доля должников=', f'{rezul:.2%}')

категория A     доля должников= 7.69%
категория B     доля должников= 6.96%
категория C     доля должников= 8.57%
категория D     доля должников= 6.00%
категория E     доля должников= 9.09%


In [84]:
grouped_purpose = data_frame.pivot_table(index='purpose_category', columns = 'debt', values='income_type', aggfunc='count') 
value_purpose=data_frame['purpose_category'].sort_values().unique() 
for i in value_purpose: 
    rezul=grouped_purpose[1][i]/(grouped_purpose[1][i]+grouped_purpose[0][i])
    print('цель кредита: ',i, '    доля должников=', f'{rezul:.2%}')

цель кредита:  операции с автомобилем     доля должников= 9.36%
цель кредита:  операции с недвижимостью     доля должников= 7.23%
цель кредита:  получение образования     доля должников= 9.22%
цель кредита:  проведение свадьбы     доля должников= 8.00%
